In [1]:
import pandas as pd
from tcrdist.repertoire import TCRrep
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_curve, auc, roc_auc_score, confusion_matrix, precision_score, recall_score, f1_score, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
pd.set_option('display.max_columns', None)
data=pd.read_csv('./vdjdb.txt',sep='\t', header=0)

data

,complex.id,gene,cdr3,v.segm,j.segm,species,mhc.a,mhc.b,mhc.class,antigen.epitope,antigen.gene,antigen.species,reference.id,method,meta,cdr3fix,vdjdb.score,web.method,web.method.seq,web.cdr3fix.nc,web.cdr3fix.unmp
0,1,TRA,CIVRAPGRADMRF,TRAV26-1*01,TRAJ43*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,Nef,HIV-1,PMID:15596521,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CIVRAPGRADMRF"", ""cdr3_old"": ""CIVRAPG...",2,sort,sanger,no,no
1,1,TRB,CASSYLPGQGDHYSNQPQHF,TRBV13*01,TRBJ1-5*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,Nef,HIV-1,PMID:15596521,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CASSYLPGQGDHYSNQPQHF"", ""cdr3_old"": ""...",2,sort,sanger,no,no
2,0,TRB,CASSFEAGQGFFSNQPQHF,TRBV13*01,TRBJ1-5*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,Nef,HIV-1,PMID:15596521,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CASSFEAGQGFFSNQPQHF"", ""cdr3_old"": ""C...",2,sort,sanger,no,no
3,2,TRA,CAVPSGAGSYQLTF,TRAV20*01,TRAJ28*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,Nef,HIV-1,PMID:15596521,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CAVPSGAGSYQLTF"", ""cdr3_old"": ""CAVPSG...",2,sort,sanger,no,no
4,2,TRB,CASSFEPGQGFYSNQPQHF,TRBV13*01,TRBJ1-5*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,Nef,HIV-1,PMID:15596521,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CASSFEPGQGFYSNQPQHF"", ""cdr3_old"": ""C...",2,sort,sanger,no,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92766,30592,TRB,CASSPGQGGDNEQFF,TRBV7-3*01,TRBJ2-1*01,HomoSapiens,HLA-DQA1*05:01,HLA-DQB1*02:01,MHCII,PQQPFPQPEQPFP,Gluten,Wheat,PMID:33927715,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD4+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CASSPGQGGDNEQFF"", ""cdr3_old"": ""CASSP...",0,sort,singlecell,no,no
92767,30593,TRA,CAPQGATNKLIF,TRAV12-2*01,TRAJ32*01,HomoSapiens,HLA-DQA1*05:01,HLA-DQB1*02:01,MHCII,PQQPFPQPEQPFP,Gluten,Wheat,PMID:33927715,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD4+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CAPQGATNKLIF"", ""cdr3_old"": ""CAPQGATN...",2,sort,singlecell,no,no
92768,30593,TRB,CASSLGAGGQETQYF,TRBV5-1*01,TRBJ2-5*01,HomoSapiens,HLA-DQA1*05:01,HLA-DQB1*02:01,MHCII,PQQPFPQPEQPFP,Gluten,Wheat,PMID:33927715,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD4+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CASSLGAGGQETQYF"", ""cdr3_old"": ""CASSL...",2,sort,singlecell,no,no
92769,30594,TRA,CLVGGSGGYNKLIF,TRAV4*01,TRAJ4*01,HomoSapiens,HLA-DQA1*05:01,HLA-DQB1*02:01,MHCII,PQQPFPQPEQPFP,Gluten,Wheat,PMID:33927715,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD4+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CLVGGSGGYNKLIF"", ""cdr3_old"": ""CLVGGS...",0,sort,singlecell,no,no


In [2]:
columns_to_drop = ['reference.id', 'method', 'meta','cdr3fix','web.method','web.method.seq','web.cdr3fix.nc','web.cdr3fix.unmp','mhc.a','mhc.b','antigen.gene']
data.drop(columns=columns_to_drop, inplace=True)
data=data.dropna()
data.head()


,complex.id,gene,cdr3,v.segm,j.segm,species,mhc.class,antigen.epitope,antigen.species,vdjdb.score
0,1,TRA,CIVRAPGRADMRF,TRAV26-1*01,TRAJ43*01,HomoSapiens,MHCI,FLKEKGGL,HIV-1,2
1,1,TRB,CASSYLPGQGDHYSNQPQHF,TRBV13*01,TRBJ1-5*01,HomoSapiens,MHCI,FLKEKGGL,HIV-1,2
2,0,TRB,CASSFEAGQGFFSNQPQHF,TRBV13*01,TRBJ1-5*01,HomoSapiens,MHCI,FLKEKGGL,HIV-1,2
3,2,TRA,CAVPSGAGSYQLTF,TRAV20*01,TRAJ28*01,HomoSapiens,MHCI,FLKEKGGL,HIV-1,2
4,2,TRB,CASSFEPGQGFYSNQPQHF,TRBV13*01,TRBJ1-5*01,HomoSapiens,MHCI,FLKEKGGL,HIV-1,2


In [3]:
data_human = data[data['species'] == 'HomoSapiens']
data_mouse = data[data['species'] == 'MusMusculus']
data_monkey = data[data['species'] == 'MacacaMulatta']
df=data_human
# # Filter data for alpha chain and beta chain
data_alpha = df[df['gene'] == 'TRA'].copy()
data_beta = df[df['gene'] == 'TRB'].copy()
# Rename data columns to match TCRrep's expected format
data_alpha.rename(columns={'cdr3': 'cdr3_a_aa', 'v.segm': 'v_a_gene', 'j.segm': 'j_a_gene', 'antigen.epitope': 'epitope'}, inplace=True)
data_beta.rename(columns={'cdr3': 'cdr3_b_aa', 'v.segm': 'v_b_gene', 'j.segm': 'j_b_gene', 'antigen.epitope': 'epitope'}, inplace=True)
data_alpha.to_csv('human_alpha.csv')
data_beta.to_csv('human_beta.csv')

In [4]:
# Merge by complex.id, assuming that each complex.id corresponds to a pair of alpha and beta chains
# Make sure complex.id is not equal to 0
data_alpha = data_alpha[data_alpha['complex.id'] != 0]
data_beta = data_beta[data_beta['complex.id'] != 0]
# Merge alpha and beta chain data
common_columns = ['species', 'epitope','antigen.species','mhc.class','vdjdb.score']
data_beta_reduced = data_beta.drop(common_columns, axis=1)
data_combined = pd.merge(data_alpha,data_beta_reduced, on='complex.id' )
data_beta.to_csv('human_combined.csv')

In [5]:
data_combined 

,complex.id,gene_x,cdr3_a_aa,v_a_gene,j_a_gene,species,mhc.class,epitope,antigen.species,vdjdb.score,gene_y,cdr3_b_aa,v_b_gene,j_b_gene
0,1,TRA,CIVRAPGRADMRF,TRAV26-1*01,TRAJ43*01,HomoSapiens,MHCI,FLKEKGGL,HIV-1,2,TRB,CASSYLPGQGDHYSNQPQHF,TRBV13*01,TRBJ1-5*01
1,2,TRA,CAVPSGAGSYQLTF,TRAV20*01,TRAJ28*01,HomoSapiens,MHCI,FLKEKGGL,HIV-1,2,TRB,CASSFEPGQGFYSNQPQHF,TRBV13*01,TRBJ1-5*01
2,4,TRA,CAYRPPGTYKYIF,TRAV38-2/DV8*01,TRAJ40*01,HomoSapiens,MHCI,FLKEKGGL,HIV-1,2,TRB,CASSALASLNEQFF,TRBV14*01,TRBJ2-1*01
3,5,TRA,CIVRAPGRADMRF,TRAV26-1*01,TRAJ43*01,HomoSapiens,MHCI,FLKEQGGL,HIV-1,2,TRB,CASSYLPGQGDHYSNQPQHF,TRBV13*01,TRBJ1-5*01
4,6,TRA,CAVPSGAGSYQLTF,TRAV20*01,TRAJ28*01,HomoSapiens,MHCI,FLKEQGGL,HIV-1,2,TRB,CASSFEPGQGFYSNQPQHF,TRBV13*01,TRBJ1-5*01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27573,30590,TRA,CMDEGGSNYKLTF,TRAV26-1*01,TRAJ53*01,HomoSapiens,MHCII,PQPELPYPQPQL,Wheat,0,TRB,CASSVRSTDTQYF,TRBV7-2*01,TRBJ2-3*01
27574,30591,TRA,CSLYNNNDMRF,TRAV26-1*01,TRAJ43*01,HomoSapiens,MHCII,PQPELPYPQPQL,Wheat,0,TRB,CASSLRYTDTQYF,TRBV7-2*01,TRBJ2-3*01
27575,30592,TRA,CALSTDSWGKLQF,TRAV6*01,TRAJ24*01,HomoSapiens,MHCII,PQQPFPQPEQPFP,Wheat,0,TRB,CASSPGQGGDNEQFF,TRBV7-3*01,TRBJ2-1*01
27576,30593,TRA,CAPQGATNKLIF,TRAV12-2*01,TRAJ32*01,HomoSapiens,MHCII,PQQPFPQPEQPFP,Wheat,2,TRB,CASSLGAGGQETQYF,TRBV5-1*01,TRBJ2-5*01


In [6]:
df=data_combined 
n_orig = df.shape[0]
print(f'The dataset contains {df.shape[0]} TCRs for {len(df["epitope"].unique())} different epitopes. The amount of TCRs per epitope:\n')
df['epitope'].value_counts()

The dataset contains 27578 TCRs for 889 different epitopes. The amount of TCRs per epitope:



epitope
KLGGALQAK          13695
GILGFVFTL           2159
AVFDRKSDAK          1730
RAKFKQLL            1226
IVTDFSVIK            724
                   ...  
GLNDNLLEI              1
AMMFTSDLA              1
IVDTVSALV              1
GQVELGGGNAVEVCK        1
TLGVLVPHV              1
Name: count, Length: 889, dtype: int64

In [7]:
#Drop duplicate TCRs for each epitope separately
drop_dup_cols=['cdr3_a_aa', 'v_a_gene', 'j_a_gene','cdr3_b_aa', 'v_b_gene', 'j_b_gene','epitope']
df.reset_index(drop=True, inplace=True)
print(f'Dropped {n_orig - df.shape[0]} duplicate TCRs by columns {drop_dup_cols}.')
print(f'The new amount of TCRs per epitope:\n')
df['epitope'].value_counts()

Dropped 0 duplicate TCRs by columns ['cdr3_a_aa', 'v_a_gene', 'j_a_gene', 'cdr3_b_aa', 'v_b_gene', 'j_b_gene', 'epitope'].
The new amount of TCRs per epitope:



epitope
KLGGALQAK          13695
GILGFVFTL           2159
AVFDRKSDAK          1730
RAKFKQLL            1226
IVTDFSVIK            724
                   ...  
GLNDNLLEI              1
AMMFTSDLA              1
IVDTVSALV              1
GQVELGGGNAVEVCK        1
TLGVLVPHV              1
Name: count, Length: 889, dtype: int64

In [8]:
#Drop epitopes with less than 60 TCRs
peptide_counts = df['epitope'].value_counts()
peptide_list = peptide_counts[peptide_counts >=60].index
df = df[df['epitope'].isin(peptide_list)]
df.reset_index(drop=True, inplace=True)

print(f'The new amount of TCRs per epitope, after dropping epitopes with less than 60 TCRs:\n')
df['epitope'].value_counts()


The new amount of TCRs per epitope, after dropping epitopes with less than 60 TCRs:



epitope
KLGGALQAK         13695
GILGFVFTL          2159
AVFDRKSDAK         1730
RAKFKQLL           1226
IVTDFSVIK           724
NLVPMVATV           553
YLQPRTFLL           464
RLRAEAQVK           422
TFEYVSQPFLMDLE      409
SPRWYFYYL           408
ELAGIGILTV          390
GLCTLVAML           387
TTDPSFLGRY          386
LLWNGPMAV           239
CINGVCWTV           231
ATDALMTGF           129
QYIKWPWYI           125
LTDEMIAQY           125
KTFPPTEPK           123
KSKRTPMGF           121
DATYQRTRALVR        100
NQKLIANQF            72
RAQAPPPSW            69
MEVTPSGTWL           68
TPRVTGGGAM           62
PKYVKQNTLKLAT        60
HPVTKYIM             60
Name: count, dtype: int64

In [10]:
df.to_csv('human_combined.csv')

In [9]:
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_curve, auc, roc_auc_score, confusion_matrix, precision_score, recall_score, f1_score, accuracy_score


In [13]:
new_data=pd.read_csv('human_combined.csv')
score_distribution = pd.pivot_table(new_data, index='epitope', columns='vdjdb.score', aggfunc='size', fill_value=0)
print(score_distribution)

vdjdb.score         0    1   2    3
epitope                            
ATDALMTGF         128    1   0    0
AVFDRKSDAK       1730    0   0    0
CINGVCWTV         210   21   0    0
DATYQRTRALVR       80   20   0    0
ELAGIGILTV        367    1  13    9
GILGFVFTL        1913  230   0   16
GLCTLVAML         317   52  16    2
HPVTKYIM           59    1   0    0
IVTDFSVIK         720    4   0    0
KLGGALQAK       13694    1   0    0
KSKRTPMGF         120    1   0    0
KTFPPTEPK         121    2   0    0
LLWNGPMAV         239    0   0    0
LTDEMIAQY         125    0   0    0
MEVTPSGTWL         68    0   0    0
NLVPMVATV         112   65  37  339
NQKLIANQF          72    0   0    0
PKYVKQNTLKLAT      56    0   0    4
QYIKWPWYI          77   47   1    0
RAKFKQLL         1222    4   0    0
RAQAPPPSW          64    5   0    0
RLRAEAQVK         422    0   0    0
SPRWYFYYL         387   13   8    0
TFEYVSQPFLMDLE    322   87   0    0
TPRVTGGGAM         53    2   5    2
TTDPSFLGRY        386    0  

In [15]:
epitopes_to_remove=['KLGGALQAK','LLWNGPMAV ','LTDEMIAQY','AVFDRKSDAK','MEVTPSGTWL','NQKLIANQF','RLRAEAQVK', 'TTDPSFLGRY','LLWNGPMAV']
filtered_data = new_data[~new_data['epitope'].isin(epitopes_to_remove)]
score_distribution = pd.pivot_table(filtered_data, index='epitope', columns='vdjdb.score', aggfunc='size', fill_value=0)
print(score_distribution)

vdjdb.score        0    1   2    3
epitope                           
ATDALMTGF        128    1   0    0
CINGVCWTV        210   21   0    0
DATYQRTRALVR      80   20   0    0
ELAGIGILTV       367    1  13    9
GILGFVFTL       1913  230   0   16
GLCTLVAML        317   52  16    2
HPVTKYIM          59    1   0    0
IVTDFSVIK        720    4   0    0
KSKRTPMGF        120    1   0    0
KTFPPTEPK        121    2   0    0
NLVPMVATV        112   65  37  339
PKYVKQNTLKLAT     56    0   0    4
QYIKWPWYI         77   47   1    0
RAKFKQLL        1222    4   0    0
RAQAPPPSW         64    5   0    0
SPRWYFYYL        387   13   8    0
TFEYVSQPFLMDLE   322   87   0    0
TPRVTGGGAM        53    2   5    2
YLQPRTFLL        400   60   1    3
